# RNN

*Recurent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。RNNは一つの層として扱うことが多い。

<u>2章: 深層学習を用いた言語モデル</u>では、深層学習を活用して、ある単語から次の単語を予測するモデルを作成した。しかし、これでは文脈を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。  
RNNを用いることで、独立していた時間ごとの演算が繋がりを持つようになり、文脈を考慮した予測が可能になる。

In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F


---

## RNNの構造

RNNはある時間$t$の入力$x_t$に対して以下のような演算を行い、出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。  
演算内容はシンプルに捉えることができて、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数$\mathrm{tanh}$に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。

実装してみるとこんな感じ。

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        z = self.fc_input(x) + self.fc_output(h)
        return F.tanh(z)

適当なパラメータで時間ごとの入力$x_t$と初めの出力$h_0$を定義し、与える。$h_0$は0ベクトルで良い。

In [10]:
batch_size, input_size, output_size = 2, 3, 4
x1 = torch.randn(batch_size, input_size)
x2 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, output_size)

rnn = RNN(input_size, output_size)
h1 = rnn(x1, h0)
h1

tensor([[ 0.6550,  0.0274, -0.2787,  0.1202],
        [ 0.4972,  0.8112, -0.8748,  0.1442]], grad_fn=<TanhBackward0>)

この$h$を用いて次の時間の演算を行う。

In [8]:
h2 = rnn(x2, h1)
h2

tensor([[ 0.6286,  0.6886,  0.4073,  0.5147],
        [-0.7758,  0.6373,  0.4783,  0.6019]], grad_fn=<TanhBackward0>)

これを繰り返すことで、時系列的な情報を保持しながら演算を行うことが出来る。

また、基本的には、モデルでの時間ごとの出力にRNNの出力$h_t$をそのまま用いることはない。$h_t$を更に幾つかの層に通して、最終的な出力を決定する。$h_t$が直接観測されることはないため、潜在変数や隠れ状態と表現される。変数名に$y$ではなく$h$を使っている理由がそこにある（隠れる=hide）。


---

## PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。↑で実装したもの。  
[RNNCell — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html)

In [21]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, '\n')

Parameter containing:
tensor([[ 0.2626,  0.4594,  0.2198],
        [-0.4493, -0.2588,  0.3801],
        [ 0.4749, -0.2739, -0.3912],
        [-0.3008, -0.1541, -0.4552]], requires_grad=True) 

Parameter containing:
tensor([[-0.4907,  0.2413,  0.3903, -0.2838],
        [-0.0134, -0.0131, -0.1561,  0.4849],
        [ 0.3408,  0.2592, -0.4880,  0.2354],
        [-0.2071, -0.1277,  0.2914, -0.0749]], requires_grad=True) 

Parameter containing:
tensor([ 0.4921, -0.1122,  0.1269, -0.4486], requires_grad=True) 

Parameter containing:
tensor([-0.2964,  0.0582, -0.3268,  0.0762], requires_grad=True) 



パラメータが4つ。

In [22]:
x1 = torch.randn(batch_size, input_size)
rnn(x1)

tensor([[ 0.5763, -0.6013,  0.0199, -0.4733],
        [-0.6790,  0.0564,  0.0685,  0.6976]], grad_fn=<TanhBackward0>)

$h$を入力しなかった場合は勝手に0ベクトルになる。

### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
[RNN — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)



In [26]:
batch_size, input_size, hidden_size, seq_length = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_length, input_size)
y, h = rnn(x)
y.shape

torch.Size([2, 5, 4])

バッチごとに全ての時間の隠れ状態が出力される。  
また`h`には最後の時間の隠れ状態が出力される。

In [27]:
h.shape

torch.Size([1, 2, 4])

初めに次元が1つ追加されるのは仕様。